In [33]:
import numpy as np
import pandas as pd
import collections

import matplotlib.pyplot as plt

% matplotlib inline

Я напрочь забыл всё, что делал с SGD классификатором. Поэтому надо бы вспомнить. Оптимальный вариант - сформировать хороший pipepline с тем, что делать одну процедуру для всех.

<ol> <u>Что необходимо сделать:</u>
<li> Скоринговая функция расчёта f1
<li> Какие признаки должны войти в dataset?
<li> функционал подбора гиперпараметров
</ol>

Для каждого пользователя следует рассчитать следующие свойства:

1. Полный список всех продуктов (fullBasket)
2. Для каждого продукта из fullBasket
    - сколько раз этот продукт был куплен;
    - средняя периодичность покупок данного товара (в днях);
    - сколько дней прошло с последней покупки данного товара;
    - 

In [4]:
# подгружаем данные
# This file tells to which set (prior, train, test) an order belongs.
# You are predicting reordered items only for the test set orders. 'order_dow' is the day of week.

# COLUMNS:
# order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
orders = pd.read_pickle('../data/orders.pcl')
ordersGroup = orders.groupby('user_id')
orders.head()

,user_id,eval_set,order_number,days_since_prior_order
order_id,,,,
2539329,1,prior,1,NaN
2398795,1,prior,2,15.0
473747,1,prior,3,21.0
2254736,1,prior,4,29.0
431534,1,prior,5,28.0


In [6]:
items = pd.read_pickle('../data/items.pcl')
itemsGroup = items.groupby('order_id')
items.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


## что теперь делаем?
1) Проходим по всем пользователям.

2) Получаем список всех prior заказов пользователя.

3) П

In [183]:
tUser = 1
userOrders = ordersGroup.get_group(tUser).fillna(0)
userOrders = userOrders.assign(date = userOrders.days_since_prior_order.cumsum()) 
userOrdersId = userOrders.index.values
priorOrders = userOrdersId[:-1]
lastOrder = userOrdersId[-1]

print priorOrders
print lastOrder

userOrders.head()

[2539329 2398795  473747 2254736  431534 3367565  550135 3108588 2295261
 2550362]
1187899


,user_id,eval_set,order_number,days_since_prior_order,date
order_id,,,,,
2539329,1,prior,1,0.0,0.0
2398795,1,prior,2,15.0,15.0
473747,1,prior,3,21.0,36.0
2254736,1,prior,4,29.0,65.0
431534,1,prior,5,28.0,93.0


In [176]:
cnt = collections.Counter()
allBaskets = [itemsGroup.get_group(ordId).product_id.values for ordId in priorOrders]
basketSize = []
for k in allBaskets:
    basketSize.append(k.shape[0])
    for v in k:
        cnt[v]+=1
tmp = pd.DataFrame.from_dict(cnt, orient='index').reset_index().rename(columns = {'index':'product_id',0:'amount'})
basketSize = np.array(basketSize)
print basketSize
print itemsGroup.get_group(lastOrder).shape[0]
tmp.head(10)

[5 6 5 5 8 4 5 6 6 9]
11


,product_id,amount
0,17122,1
1,41787,1
2,14084,1
3,26405,2
4,26088,2
5,39657,1
6,12427,10
7,25133,8
8,35951,1
9,38928,1


In [181]:
[itemsGroup.get_group(ordId) for ordId in priorOrders]

[          order_id  product_id
 24076664   2539329         196
 24076665   2539329       14084
 24076666   2539329       12427
 24076667   2539329       26088
 24076668   2539329       26405,           order_id  product_id
 22742744   2398795         196
 22742745   2398795       10258
 22742746   2398795       12427
 22742747   2398795       13176
 22742748   2398795       26088
 22742749   2398795       13032,          order_id  product_id
 4488095    473747         196
 4488096    473747       12427
 4488097    473747       10258
 4488098    473747       25133
 4488099    473747       30450,           order_id  product_id
 21376074   2254736         196
 21376075   2254736       12427
 21376076   2254736       10258
 21376077   2254736       25133
 21376078   2254736       26405,          order_id  product_id
 4089398    431534         196
 4089399    431534       12427
 4089400    431534       10258
 4089401    431534       25133
 4089402    431534       10326
 4089403    431534  